### Çalışma Verileri Çekme ve Düzenleme

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('Çalışma Çizelgesi v2.xlsx')
df.head()

,Gün,Ay,Yıl,GünAdı,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay
0,5,12,2022,Pazartesi,5,6,0.833333,0,0
1,6,12,2022,Salı,3,6,0.500000,0,0
2,7,12,2022,Çarşamba,0,0,0.000000,0,1
3,8,12,2022,Perşembe,0,6,0.000000,0,0
4,9,12,2022,Cuma,0,6,0.000000,0,0


In [2]:
df['ÇalışmaOranı'] = np.round( ( df['ÇalışmaOranı'] * 100 ) , 1 )
df['Yapilmayan'] = [ 1 if a > 0 and b == 0 else 0 for a,b in zip(df['Hedef'] , df['Yapılan'])]

In [3]:
df['Tarih'] = pd.to_datetime(df['Yıl'].astype(str) + '-' + df['Ay'].astype(str) + '-' + df['Gün'].astype(str))
df['Tarih'] = df['Tarih'].dt.strftime('%d-%m-%Y')
df.drop(columns=['Gün', 'Ay','Yıl'], inplace=True)
df = df[['Tarih', 'GünAdı', 'Yapılan', 'Hedef', 'ÇalışmaOranı', 'FreeDay', 'OffDay', 'Yapilmayan']]

### Günlük Çalışma

#### Günlük Toplam

In [4]:
day_count = df['GünAdı'].value_counts().reset_index()
day_count.columns = ['GünAdı', 'GünSayı']

toplam_calisma = df.groupby('GünAdı')[['Yapılan','Hedef', 'FreeDay', 'OffDay', 'Yapilmayan']].sum().reindex(['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']).reset_index()
toplam_calisma['Çalışma %'] = np.round( ((toplam_calisma['Yapılan'] / toplam_calisma['Hedef']) * 100) ,1) 
toplam_calisma['BoşGünler'] = toplam_calisma['FreeDay'] + toplam_calisma['OffDay'] + toplam_calisma['Yapilmayan']
toplam_calisma = toplam_calisma[['GünAdı', 'Yapılan', 'Hedef', 'Çalışma %', 'FreeDay', 'OffDay', 'Yapilmayan', 'BoşGünler']]
toplam_calisma['Boş %'] = np.round( ((toplam_calisma['BoşGünler'] / df.shape[0]) * 100) ,1) 

toplam_calisma = toplam_calisma.merge(day_count, on='GünAdı', how='left')
toplam_calisma

,GünAdı,Yapılan,Hedef,Çalışma %,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %,GünSayı
0,Pazartesi,673,1192,56.5,2,10,6,18,1.7,148
1,Salı,530,1122,47.2,3,13,7,23,2.2,148
2,Çarşamba,513,1128,45.5,2,18,10,30,2.9,147
3,Perşembe,478,1040,46.0,2,24,9,35,3.4,147
4,Cuma,389,984,39.5,3,31,19,53,5.1,147
5,Cumartesi,94,194,48.5,99,14,4,117,11.3,147
6,Pazar,74,197,37.6,103,13,7,123,11.9,147


#### Günlük Ortalama

In [5]:
ortalama_gunluk = df.groupby('GünAdı')[['Yapılan','Hedef', 'FreeDay', 'OffDay', 'Yapilmayan']].mean().reindex(['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']).reset_index()
ortalama_gunluk['Çalışma %'] = np.round( ((ortalama_gunluk['Yapılan'] / ortalama_gunluk['Hedef']) * 100) ,1) 
ortalama_gunluk = ortalama_gunluk.round(1)
ortalama_gunluk

,GünAdı,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Çalışma %
0,Pazartesi,4.5,8.1,0.0,0.1,0.0,56.5
1,Salı,3.6,7.6,0.0,0.1,0.0,47.2
2,Çarşamba,3.5,7.7,0.0,0.1,0.1,45.5
3,Perşembe,3.3,7.1,0.0,0.2,0.1,46.0
4,Cuma,2.6,6.7,0.0,0.2,0.1,39.5
5,Cumartesi,0.6,1.3,0.7,0.1,0.0,48.5
6,Pazar,0.5,1.3,0.7,0.1,0.0,37.6


### ilk 6 aylık periyot [ Ocak - Haziran ] ayları çalışma Analizi

In [6]:
df = pd.read_excel('Çalışma Çizelgesi v2.xlsx')
df['Tarih'] = pd.to_datetime(df['Yıl'].astype(str) + '-' + df['Ay'].astype(str) + '-' + df['Gün'].astype(str))
df['Tarih'] = df['Tarih'].dt.strftime('%d-%m-%Y')
df.drop(columns=['Gün', 'Ay','Yıl'], inplace=True)
df['Yapilmayan'] = [ 1 if a > 0 and b == 0 else 0 for a,b in zip(df['Hedef'] , df['Yapılan'])]
df['ÇalışmaOranı'] = np.round( ( df['ÇalışmaOranı'] * 100 ) , 1 )
df = df[['Tarih', 'GünAdı', 'Yapılan', 'Hedef', 'ÇalışmaOranı', 'FreeDay', 'OffDay', 'Yapilmayan']]


In [7]:
df.head()

,Tarih,GünAdı,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan
0,05-12-2022,Pazartesi,5,6,83.3,0,0,0
1,06-12-2022,Salı,3,6,50.0,0,0,0
2,07-12-2022,Çarşamba,0,0,0.0,0,1,0
3,08-12-2022,Perşembe,0,6,0.0,0,0,1
4,09-12-2022,Cuma,0,6,0.0,0,0,1


In [8]:
df['Tarih'] = pd.to_datetime(df['Tarih'], format='%d-%m-%Y')
df_2023_1_2 = df.loc[(df['Tarih'] >= '2023-01-01') & (df['Tarih'] <= '2023-06-30')]
df_2024_1_2 = df.loc[(df['Tarih']>='2024-01-01') & (df['Tarih']<='2024-06-30')]
df_2025_1_2 = df.loc[(df['Tarih']>='2025-01-01') & (df['Tarih']<='2025-06-30')]
df_2023_1_2

,Tarih,GünAdı,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan
27,2023-01-01,Pazar,0,0,0.0,0,1,0
28,2023-01-02,Pazartesi,3,4,75.0,0,0,0
29,2023-01-03,Salı,6,6,100.0,0,0,0
30,2023-01-04,Çarşamba,3,6,50.0,0,0,0
31,2023-01-05,Perşembe,7,6,116.7,0,0,0
...,...,...,...,...,...,...,...,...
203,2023-06-26,Pazartesi,0,0,0.0,0,1,0
204,2023-06-27,Salı,0,0,0.0,0,1,0
205,2023-06-28,Çarşamba,0,0,0.0,0,1,0
206,2023-06-29,Perşembe,0,0,0.0,0,1,0


In [9]:
df_2023_1_2 = df_2023_1_2.groupby(df_2023_1_2['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_1_2 = df_2024_1_2.groupby(df_2024_1_2['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_1_2 = df_2025_1_2.groupby(df_2025_1_2['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})


In [10]:
df_2023_1_2['Calisma%'] = np.round( ((df_2023_1_2['Yapılan'] / df_2023_1_2['Hedef']) * 100) ,1)
df_2023_1_2

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Calisma%
Tarih,,,,,,
2023,570,1014,30,33,8,56.2


In [11]:
df_2024_1_2['Calisma%'] = np.round( ((df_2024_1_2['Yapılan'] / df_2024_1_2['Hedef']) * 100) ,1)
df_2024_1_2

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Calisma%
Tarih,,,,,,
2024,510,976,48,9,6,52.3


In [12]:
df_2025_1_2['Calisma%'] = np.round( ((df_2025_1_2['Yapılan'] / df_2025_1_2['Hedef']) * 100) ,1)
df_2025_1_2

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Calisma%
Tarih,,,,,,
2025,530,1165,52,14,4,45.5


### 3 aylık periyotlara göre performansların analizi

Ocak - Şubat - Mart -> 1_4

Nisan - Mayıs - Haziran -> 2_4

Temmuz - Ağustos - Eylül -> 3_4

Ekim - Kasım - Aralık -> 4_4

In [13]:
df_2023_1_4 = df.loc[(df['Tarih'] >= '2023-01-01') & (df['Tarih'] <= '2023-03-31')]
df_2023_2_4 = df.loc[(df['Tarih'] >= '2023-04-01') & (df['Tarih'] <= '2023-06-30')]
df_2023_3_4 = df.loc[(df['Tarih'] >= '2023-07-01') & (df['Tarih'] <= '2023-09-30')]
df_2023_4_4 = df.loc[(df['Tarih'] >= '2023-10-01') & (df['Tarih'] <= '2023-12-31')]

df_2024_1_4 = df.loc[(df['Tarih'] >= '2024-01-01') & (df['Tarih'] <= '2024-03-31')]
df_2024_2_4 = df.loc[(df['Tarih'] >= '2024-04-01') & (df['Tarih'] <= '2024-06-30')]
df_2024_3_4 = df.loc[(df['Tarih'] >= '2024-07-01') & (df['Tarih'] <= '2024-09-30')]
df_2024_4_4 = df.loc[(df['Tarih'] >= '2024-10-01') & (df['Tarih'] <= '2024-12-31')]

df_2025_1_4 = df.loc[(df['Tarih'] >= '2025-01-01') & (df['Tarih'] <= '2025-03-31')]
df_2025_2_4 = df.loc[(df['Tarih'] >= '2025-04-01') & (df['Tarih'] <= '2025-06-30')]
df_2025_3_4 = df.loc[(df['Tarih'] >= '2025-07-01') & (df['Tarih'] <= '2025-09-30')]

In [14]:
df_2023_1_4 = df_2023_1_4.groupby(df_2023_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_2_4 = df_2023_2_4.groupby(df_2023_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_3_4 = df_2023_3_4.groupby(df_2023_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_4_4 = df_2023_4_4.groupby(df_2023_4_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_1_4 = df_2024_1_4.groupby(df_2024_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_2_4 = df_2024_2_4.groupby(df_2024_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_3_4 = df_2024_3_4.groupby(df_2024_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_4_4 = df_2024_4_4.groupby(df_2024_4_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_1_4 = df_2025_1_4.groupby(df_2025_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_2_4 = df_2025_2_4.groupby(df_2025_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_3_4 = df_2025_3_4.groupby(df_2025_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})

df_2023_1_4['Donem'] = '2023_1_4'
df_2023_2_4['Donem'] = '2023_2_4'
df_2023_3_4['Donem'] = '2023_3_4'
df_2023_4_4['Donem'] = '2023_4_4'
df_2024_1_4['Donem'] = '2024_1_4'
df_2024_2_4['Donem'] = '2024_2_4'
df_2024_3_4['Donem'] = '2024_3_4'
df_2024_4_4['Donem'] = '2024_4_4'
df_2025_1_4['Donem'] = '2025_1_4'
df_2025_2_4['Donem'] = '2025_2_4'
df_2025_3_4['Donem'] = '2025_3_4'

df_2023_1_4['Calisma%'] = np.round( ((df_2023_1_4['Yapılan'] / df_2023_1_4['Hedef']) * 100) ,1)
df_2023_2_4['Calisma%'] = np.round( ((df_2023_2_4['Yapılan'] / df_2023_2_4['Hedef']) * 100) ,1)
df_2023_3_4['Calisma%'] = np.round( ((df_2023_3_4['Yapılan'] / df_2023_3_4['Hedef']) * 100) ,1)
df_2023_4_4['Calisma%'] = np.round( ((df_2023_4_4['Yapılan'] / df_2023_4_4['Hedef']) * 100) ,1)
df_2024_1_4['Calisma%'] = np.round( ((df_2024_1_4['Yapılan'] / df_2024_1_4['Hedef']) * 100) ,1)
df_2024_2_4['Calisma%'] = np.round( ((df_2024_2_4['Yapılan'] / df_2024_2_4['Hedef']) * 100) ,1)
df_2024_3_4['Calisma%'] = np.round( ((df_2024_3_4['Yapılan'] / df_2024_3_4['Hedef']) * 100) ,1)
df_2024_4_4['Calisma%'] = np.round( ((df_2024_4_4['Yapılan'] / df_2024_4_4['Hedef']) * 100) ,1)
df_2025_1_4['Calisma%'] = np.round( ((df_2025_1_4['Yapılan'] / df_2025_1_4['Hedef']) * 100) ,1)
df_2025_2_4['Calisma%'] = np.round( ((df_2025_2_4['Yapılan'] / df_2025_2_4['Hedef']) * 100) ,1)
df_2025_3_4['Calisma%'] = np.round( ((df_2025_3_4['Yapılan'] / df_2025_3_4['Hedef']) * 100) ,1)




In [15]:
df_2023_1_4.set_index('Donem', inplace=True)
df_2023_2_4.set_index('Donem', inplace=True)
df_2023_3_4.set_index('Donem', inplace=True)
df_2023_4_4.set_index('Donem', inplace=True)
df_2024_1_4.set_index('Donem', inplace=True)
df_2024_2_4.set_index('Donem', inplace=True)
df_2024_3_4.set_index('Donem', inplace=True)
df_2024_4_4.set_index('Donem', inplace=True)
df_2025_1_4.set_index('Donem', inplace=True)
df_2025_2_4.set_index('Donem', inplace=True)
df_2025_3_4.set_index('Donem', inplace=True)

In [16]:
merged_df = pd.concat([
    df_2023_1_4, df_2023_2_4, df_2023_3_4, df_2023_4_4,
    df_2024_1_4, df_2024_2_4, df_2024_3_4, df_2024_4_4,
    df_2025_1_4, df_2025_2_4, df_2025_3_4
])

In [17]:
merged_df

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Calisma%
Donem,,,,,,
2023_1_4,251,396,20,18,4,63.4
2023_2_4,319,618,10,15,4,51.6
2023_3_4,61,182,8,37,20,33.5
2023_4_4,200,394,25,8,8,50.8
2024_1_4,259,496,24,2,4,52.2
2024_2_4,251,480,24,7,2,52.3
2024_3_4,314,718,12,9,0,43.7
2024_4_4,258,670,25,4,4,38.5
2025_1_4,301,629,24,6,1,47.9


In [19]:
import plotly.express as px

fig = px.line(merged_df.reset_index(), 
              x='Donem', 
              y='Calisma%',
              title='Dönemlere Göre Çalışma Yüzdesi',
              labels={'Calisma%': 'Çalışma %', 'Donem': 'Dönem'},
              markers=True)

fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False
)

fig.show()